In [ ]:
import numpy as np
import math
import scipy.linalg as la
import matplotlib.pyplot as plt
import scipy
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget


In [ ]:
def plot3D(Y, Xgrid, tgrid):
     T, X = np.meshgrid(np.transpose(tgrid), Xgrid)

     # Create a 3D plot
     fig = plt.figure(figsize=(10, 10))  # You can adjust the size as needed
     ax = fig.add_subplot(111, projection='3d')
     ax.plot_surface(T, X, np.transpose(Y))

     ax.set_xlabel('Time')
     ax.set_ylabel('Space')
     ax.set_zlabel('Value')

     plt.show()

In [ ]:
def pulse(x):
    return np.exp(-100*(x-0.5)**2)


\begin{gather*}
    u_{t}+uu_{x}=d\cdot u_{xx}\\
    u(t+\Delta t,x)\approx u(t,x)+\Delta tu_{t}+\frac{\Delta t^{2}}{2!}u_{tt}\\
    \text{Invicid Burgers equation: }u_{t}=-uu_{x}\\
    u_{tt}=-u_{t}u_{x}-uu_{xt}\\
    u_{tx}=-u_{x}^{2}-uu_{xx}\\
    u_{tt}=-uu_{x}^{2}-u(-u_{x}^{2}-uu_{xx})\\
    u_{tt}=u^{2}u_{xx} \\

    \text{LW?: } u(t+\Delta t,x)\approx u(t,x)-\Delta tuu_{x}+\frac{\Delta t^{2}}{2!}u^{2}u_{xx}\\

\end{gather*}


In [ ]:
def LWsolver(g0, N, M, a, d):
    deltaX = 1/N  # rumssteg från x0 till xN-1
    dt = 1/M  # tidsteg från t0 till tN
    Xgrid = np.linspace(0, 1, N+1)
    tgrid = np.linspace(0, 1, M+1)

    T = la.toeplitz([-2, 1] + [0] * (N-1)) ##Skapar en Toeplitz matris för disc av andraderivatan
    T[-1,0] = 1
    T[0,-1] = 1
    T = T/deltaX**2
    
    Xop = d*T*dt/2
    op = np.linalg.inv(np.eye(N+1)-Xop)@(np.eye(N+1)+Xop)

    Y = np.zeros((M+1, N+1)) #Första värdet är tidssteget, andra är x-steget.
    Y[0,:] = g0 #Sätter initialvärden
    u = np.zeros_like(Y)
    u[0,:] = g0
    LW = np.zeros(N+1)

    for n in range(M):
        Y[n+1,:] = op@np.transpose(Y[n, :])
        for j in range(N+1):
            if j == N:
                LW[j] = u[n,j] + (dt*u[n,j]*(u[n,0]-u[n,j-1])/(2*deltaX)) - (dt**2/(2*deltaX**2))*(u[n,0]-2*u[n,j]+u[n,j-1])*u[n,j]**2
            else:
                LW[j] = u[n,j] + (dt*u[n,j]*(u[n,j+1]-u[n,j-1])/(2*deltaX)) - (dt**2/(2*deltaX**2))*(u[n,j+1]-2*u[n,j]+u[n,j-1])*u[n,j]**2 #ingen matris
        u[n+1,:] = np.linalg.inv(np.eye(N+1)-Xop)@np.transpose(LW) + Y[n+1,:]
        #Y[i+1,-1] = Y[i+1,0]
    return Y, Xgrid, tgrid

In [ ]:
a = 1
d = 0.1
if d == 0:
     Pe = math.inf
else:
    Pe = abs(a/d)
print('Pe är: '+ str(Pe))
N, M = 10, 10

g0 = pulse(np.linspace(0, 1, N+1))

Y, Xgrid, tgrid = LWsolver(g0, N, M, a, d)
plot3D(Y, Xgrid, tgrid)
